---

# Main chat completion function

---

Same function from tutorial, with some additions like extra hyperparams

In [87]:
# %% Minimal setup
# If needed (uncomment in a notebook):
# !pip install requests python-dotenv

import os, json, textwrap, re, time
import requests

API_KEY  = os.getenv("OPENAI_API_KEY", "cse476")
API_BASE = os.getenv("API_BASE", "http://10.4.58.53:41701/v1")  
MODEL    = os.getenv("MODEL_NAME", "bens_model")              

def call_model_chat_completions(prompt: str,
                                system: str = "You are a helpful assistant. Reply with only the final answer—no explanation.",
                                model: str = MODEL,
                                temperature: float = 0.3,
                                timeout: int = 60,
                                max_tokens: int = 128,
                                top_p: int = None,
                                top_k: int = None,
                                stop: str = None) -> dict:
    """
    Calls an OpenAI-style /v1/chat/completions endpoint and returns:
    { 'ok': bool, 'text': str or None, 'raw': dict or None, 'status': int, 'error': str or None, 'headers': dict }
    """
    url = f"{API_BASE}/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type":  "application/json",
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system},
            {"role": "user",   "content": prompt}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
        "top_p": top_p,
        "top_k": top_k,
        "stop": stop
    }

    try:
        #{'id': 'chatcmpl-88b6d7e18a5542b5bed5bf2828f0661e', 'object': 'chat.completion', 'created': 1763204718, 'model': 'bens_model', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'US Highway 281', 'refusal': None, 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': [], 'reasoning_content': None}, 'logprobs': None, 'finish_reason': 'stop', 'stop_reason': None, 'token_ids': None}], 'service_tier': None, 'system_fingerprint': None, 'usage': {'prompt_tokens': 50, 'total_tokens': 57, 'completion_tokens': 7, 'prompt_tokens_details': None}, 'prompt_logprobs': None, 'prompt_token_ids': None, 'kv_transfer_params': None}
        resp = requests.post(url, headers=headers, json=payload, timeout=timeout)
        status = resp.status_code
        hdrs   = dict(resp.headers)
        if status == 200:
            data = resp.json()
            #print(data)
            text = data.get("choices", [{}])[0].get("message", {}).get("content", "")
            tokens_used = data.get("usage",[{}]).get("completion_tokens", {})
            #print('used tokens:', tokens_used)
            
            return {"ok": True, "text": text, "raw": data, "status": status, "error": None, "headers": hdrs, "tokens_used":tokens_used}
        else:
            # try best-effort to surface error text
            err_text = None
            try:
                err_text = resp.json()
            except Exception:
                err_text = resp.text
            return {"ok": False, "text": None, "raw": None, "status": status, "error": str(err_text), "headers": hdrs}
    except requests.RequestException as e:
        return {"ok": False, "text": None, "raw": None, "status": -1, "error": str(e), "headers": {}}


---
# TESTS

---

In [4]:
from IPython.display import Markdown, display

Get all the tests, load/save, define possible types

In [7]:
import json
from pprint import pprint
import random
from collections import Counter

POSSIBLE_TYPES = ['math', 'common_sense', 'planning', 'coding', 'future_prediction']

def load_save_json(path_in="parsed_dev_data.json", path_out=None, data_in=None, clear=False):
    data = json.load(open(path_in, "r", encoding="utf-8")) if not clear else []
    if path_out is not None:
        data.append(data_in)
        with open(path_out, "w") as f:
            json.dump(data, f, indent=4)
            
    return data
            
all_tests = load_save_json()

type_counts = Counter(t['domain'] for t in all_tests)
print(type_counts)

formatted_tests = []
for i, t in enumerate(all_tests, start=1):
    
    formatted_tests.append({
        "id": t['id'], # domain_domainIndex_domainTestIndex_testIndex
        "type": t['domain'],
        "prompt": t['input'],
        "expected": t['output'],
        "char_count": t['input_char_count'],
        "exp_word_count": t['exp_word_count']
    })
    
all_tests = formatted_tests

Counter({'common_sense': 400, 'math': 300, 'coding': 100, 'future_prediction': 100, 'planning': 100})


Generic get test function, mostly used for getting random tests of certain type(s)

In [ ]:
def print_json(test):
    print(json.dumps(test, indent=2, ensure_ascii=False))

#pass test_type as a list of types
#generalized get test function
def get_tests(n=0, test_type=POSSIBLE_TYPES, start=0, end=None, lower_char=0, upper_char=float('inf'), lower_exp=0, upper_exp=float('inf'), seed=None, index=None):
    if index is not None: return all_tests[index]
    
    filtered_tests = [t for t in all_tests if t['type'] in test_type and lower_char <= t['char_count'] <= upper_char and lower_exp <= t['exp_word_count'] <= upper_exp]
    #print('filtered size:', len(filtered_tests))
    sample_size = min(n, len(filtered_tests))
    
    if seed is not None: random.seed(seed)
    
    if n == 0:
        return [filtered_tests[start:end]]
    elif n == -1:
        filtered_type_counts = Counter(t['type'] for t in filtered_tests)
        each_test = []
        count = 0
        
        for val in filtered_type_counts.values():
            rand = random.randint(count, count + val)
            count = count + val
            each_test.append(filtered_tests[rand])
            
        #print("sampled size:", len(each_test))    
        return each_test
    else:
        return random.sample(filtered_tests, sample_size)
    

" def get_test_type(test_type, start=0, end=None, lower=0, upper=float('inf')):\n    tests = [t for t in all_tests if t['type'] in test_type and lower <= t['char_count'] <= upper]\n    return tests[start:end]\n\ndef get_random_tests(n=5, lower=0, upper=float('inf'), test_type=POSSIBLE_TYPES):\n    filtered_tests = get_test_type(test_type=test_type, lower=lower, upper=upper) #[t for t in all_tests if lower <= t['char_count'] <= upper]\n    sample_size = min(n, len(filtered_tests)) #prevent error\n    return random.sample(filtered_tests, sample_size) "

In [ ]:
""" def interactive_chat():
    messages = ["<Start of message history>"]
    count = 0
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            print("Exiting chat.")
            break
        response = call_model_chat_completions(prompt=f"Old messages{messages}, CURRENT USER INPUT:{user_input} <--- ANSWER THIS QUESTION", temperature=0.7)
        count += 1
        messages.append(f"MESSAGE_{count}_[previous user input: {user_input}, previous system response: {response['text']}]")
        if response["ok"]:
            print("Model:", response["text"].strip())
        else:
            print("Error:", response["error"])
        print(messages) """
#interactive_chat()

### Handle guessing the prompt domain/type

In [ ]:
import re

Terrible by itself (This qwen model is just not very capable...), might work better with this new implementation to supplement low NB confidence

In [248]:
#performs really really bad
def guess_test_type(input, p1, p2):
    sys = """
    You are a basic decider model.
    Your goal is to distinguish between if an input sentence represents a math, common sense, planning, or coding question.
    Respond with one word depending on the input sentence type
    """
    #POSSIBLE_TYPES = ['math', 'common_sense', 'planning', 'coding', 'future_prediction']
    prompt = """
    What question type does the following input correspond to?
    
    INPUT: {input}
    ----------------------
    Is it a {p1} question?
    
    OR
    
    Is it a {p2} question?
    
    Respond with one word
    """
    return call_model_chat_completions(system=sys, prompt=prompt, temperature=0.3)["text"]

Simple NB classifer trained in colab on the dev data. Good test accuracy around 95.5%

In [249]:
import pickle

with open('nb_classifier.pkl', 'rb') as f:
    classifier = pickle.load(f)

with open('nb_vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)


In [250]:
data = []
with open('cse_476_final_project_test_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [251]:
def classify_prompt(prompt):
    prompt_vec = vectorizer.transform([prompt])
    
    prediction = classifier.predict(prompt_vec)[0]
    probabilities = classifier.predict_proba(prompt_vec)[0]
    
    print(f"prediction: {prediction}")
    print(f"Classes order: {classifier.classes_}")
    print(f"probabilities: {probabilities}")
    
    result = {
        'prompt': prompt,
        'predicted_type': prediction,
        'probabilities': {class_name: prob for class_name, prob in zip(classifier.classes_, probabilities)}
    }
    
    high_prob_classes = [class_name for class_name, prob in zip(classifier.classes_, probabilities) if prob >= 0.40]
    
    if len(high_prob_classes) == 2:
        print(f"TWO HIGH PROB: {high_prob_classes[0]} {high_prob_classes[1]}")
        return guess_test_type(prompt, high_prob_classes[0], high_prob_classes[1])
    
    return result['predicted_type']

""" for test,i in zip(data,range(0,100)):
    print(test)
    #print('ACTUAL:', test["domain"])
    result = classify_prompt(test["input"])
    print(f"PREDICTED: {result['predicted_type']}")
    print()
    print(f"Probabilities: {result['probabilities']}") """

' for test,i in zip(data,range(0,100)):\n    print(test)\n    #print(\'ACTUAL:\', test["domain"])\n    result = classify_prompt(test["input"])\n    print(f"PREDICTED: {result[\'predicted_type\']}")\n    print()\n    print(f"Probabilities: {result[\'probabilities\']}") '

---

# First Final Project Implementation

---

So I didn't realize we don't get expected answers on the final test data, my first implementation of the final project was based off the tutorial notebooks using expected answer.

### Self Evaluate functions

In [13]:
def self_evaluate(question, prediction, expected_answer, model=MODEL):
    """
    Use the model itself as a strict grader.
    Returns True if the model says the prediction matches the expected answer; else False.
    Falls back to a simple normalized string compare if the model's reply is malformed.
    """

    system = "You are a strict grader. Reply with exactly True or False. No punctuation. No explanation."
    prompt = f"""You are grading a question-answer pair.

Return exactly True if the PREDICTION would be accepted as correct for the EXPECTED_ANSWER.
Otherwise, return False.

QUESTION:
{question}

PREDICTION:
{prediction}

EXPECTED_ANSWER:
{expected_answer}

Answer with exactly: True or False
"""

    r = call_model_chat_completions(
        prompt,
        system=system,
        model=model,
        temperature=0.3,
    )

    reply = (r.get("text") or "").strip().lower()
    if reply.startswith("true"):
        return True
    if reply.startswith("false"):
        return False

    # No Fallback yet


In [14]:
def self_evaluate2(question, model_output, prediction, expected_answer, model=MODEL):
    """
    Use the model itself as a strict grader.
    Returns True if the model says the prediction matches the expected answer; else False.
    Falls back to a simple normalized string compare if the model's reply is malformed.
    """
    import re

    system = "You are a strict grader. Reply with exactly Yes or No. No punctuation. No explanation."
    prompt = f"""MODEL_1 thinks this ANSWER is {prediction}, do you agree with MODEL_1 decision?

QUESTION:
{question}

ANSWER:
{model_output}

EXPECTED_ANSWER:
{expected_answer}

-----------------------
MODEL_1 OUTPUT:
{prediction}
-----------------------

Answer with exactly: Yes or No. Do you agree with MODEL_1?
"""

    r = call_model_chat_completions(
        prompt,
        system=system,
        model=model,
        temperature=0.3,
    )

    reply = (r.get("text") or "").strip().lower()
    if reply.startswith("true") or reply.startswith("yes"):
        return True
    if reply.startswith("false") or reply.startswith("no"):
        return False

    # No Fallback yet


### Eval util functions

In [15]:
tf_map = {'yes':'true', 'no':'false'}
def map_tf(output, exp):
    exp = str(exp)
    exp = exp.lower().strip('.')
    out = output.lower().strip('.')
    
    #rare case when exp is actually yes/now and model output is true/false
    if exp == "yes" and out == "true": return "yes"
    if exp == "no" and out == "false": return "no"
    
    return tf_map.get(out) if out in tf_map else output

In [16]:

def basic_match_check(test, output):
    exp = test["expected"]
    
    output = map_tf(output, exp)
    
    matches = re.findall(re.escape(str(exp)), output, re.IGNORECASE)
    
    num_matches = len(matches)
    if num_matches > 0:
        #print('MATCH(ES) FOUND:', matches)
        return True
    
    #print('NO MATCH FOUND')
    return False

In [17]:
def seperator(text, tokens_used=None, max_tokens=400):
    if tokens_used is not None:
        print(f'{text} (TOKENS USED: {tokens_used}/{max_tokens})')
        if int(tokens_used) == max_tokens:
            print('MAXED TOKENS REACHED - OUTPUT TRUNCATED')
            return False
    else:
        print(text)
    print('-'*32)
    
    return True

In [18]:
def check_correct(bool1, bool2):
    correctness = bool1 and bool2
    agreement = bool1 == bool2
    
    print('✅ CORRECT') if correctness else print('❌ INCORRECT')
    print('🆗 AGREED') if agreement else print('🆘 DISAGREED')
    
    return correctness, agreement

Bunch of functions used when we have an expected answer

In [19]:
import math

def create_matches(toCount, toMatch):
    counter = Counter(toCount)
    match_counts = {word: counter.get(word, 0) for word in toMatch}
    total_matches = sum(match_counts.values())
    output_len = len(toCount)
    #print(f"{total_matches}/{output_len} : {(total_matches / output_len) * 100 if output_len != 0 else 0}%")
    #print('match counts:', match_counts)
    return total_matches, output_len

def get_cosine(expected_counter, output_counter):
    dot_product = sum(expected_counter[word] * output_counter.get(word, 0) for word in expected_counter)
    
    #print(f"Dot product: {dot_product}")
    
    exp_mag = math.sqrt(sum(v**2 for v in expected_counter.values()))
    out_mag = math.sqrt(sum(v**2 for v in output_counter.values()))
    
    cosine_sim = 0
    if exp_mag > 0 and out_mag > 0:
        cosine_sim = dot_product / (exp_mag * out_mag)
        print(f"\n[Cosine similarity: {cosine_sim}]")
        
    return cosine_sim

def get_start_end_matches(expected, output, exp_len, out_len):
    start_matches = False
    end_matches = False
    if expected[0] in output[0]: start_matches = True
    if expected[exp_len-1] in output[out_len-1]: end_matches = True
    #print('exp', expected)
    #print('output', output)
    
    #print(f"expected[0] {expected[0]}, output[0] {output[0]}")
    #print(f"expected[exp_len-1] {expected[exp_len-1]}, output[out_len-1] {output[out_len-1]}")
    #print(f"START {start_matches} END {end_matches}")
    
    return start_matches, end_matches
    
def super_match(test, output):
    expected = str(test["expected"]).replace('$', '').lower().split()
    output = output.replace('$', '').lower().split()
    
    expected_counter = Counter(expected)
    output_counter = Counter(output)
    
    #not very helpful in the long run...
    get_cosine(expected_counter, output_counter)
    
    exp_matches, out_len = create_matches(output, expected)
    out_matches, exp_len = create_matches(expected, output)
    
    return get_start_end_matches(expected, output, exp_len, out_len)
    #return match_counts

### Main first implementation agent loop

In [ ]:
def self_evaluate_tests(tests, model=MODEL, grader_model=None, sleep_sec=0.2, verbose=True):
    """
    Run the tests by querying the model for each prompt, then use LLM-as-a-judge
    (self_evaluate) to determine correctness.

    Args:
        tests: list of dicts with keys: id, prompt, expected (and optionally type)
        model: model used to generate predictions
        grader_model: model used to judge correctness (defaults to `model` if None)
        sleep_sec: small delay between calls to be polite to the API
        verbose: if True, print a summary line per test

    Returns:
        rows: list of dicts with fields:
              id, expected, got, correct, status, error
    """
    import time

    judge_model = grader_model or model
    MAX_TOKENS = 400
    final_answers = []
    count = 0
    test_samples = {
        "count": len(tests),
        "seed": None,
        "samples": None
    }
    
    for t in tests:
        sample = {
            "test_count": count,
            "id": t["id"],
            "input": t['prompt'],
            "expected": t["expected"],
            "got": None,
            "history": {
                "check_correct1": {
                    "match_check": None,
                    "self_eval": None,
                    "correctness": None,
                    "agreement": None
                },
                "no_output": False,
                "truncated": False,
                "check_correct2": {
                    "self_eval": None,
                    "self_eval2": None,
                    "correctness": None,
                    "agreement": None
                },
                "check_correct3": {
                    "self_eval2": None,
                    "sides_matching": None,
                    "correctness": None,
                    "agreement": None
                },
                "final_correctness": None
            }
        }
        count += 1
        # 1) Get model prediction
        #print('prompt:', t['prompt'])
        print('\n','='*64)
        seperator('TEST_CASE')
        print_json(t)
        #handle_test(t)
        
        r = call_model_chat_completions(
            f"{t['prompt']}",
            system="Give a short answer to each prompt, don't explain.",
            model=model,
            temperature=0.3,
            max_tokens=MAX_TOKENS
        )
        got = (r.get("text") or "").strip()
        sample["got"]=got
        tokens_used = r.get("tokens_used")
        

        got = map_tf(got, t["expected"])
        
        has_output  = True if got != "" else False
        
        #If output is truncated and both evals return true, return false
        not_truncated = seperator('\nMODEL_OUTPUT', tokens_used, MAX_TOKENS)
        
        
        got = handle_test(t, got)
        display(Markdown(f"\n{got}"))
        
        print_json(got)
        #print(got)
        #print('raw: ', got)
        
        if not not_truncated:
            #final_answers.append(False)
            sample["history"]['truncated'] = True
            print("❌ MAX TOKENS REACHED, OUTPUT TRUNCATED, SKIPPING TESTCASE ❌")
            continue
        elif has_output == False:
            sample["history"]['no_output'] = True
            print("❌ NO OUTPUT, PROMPT IS PROBABLY TOO LARGE, SKIPPING TESTCASE ❌")
            continue
        
        match_check = basic_match_check(t, got)
        match_check = bool(match_check)
        sample["history"]["check_correct1"]["match_check"] = match_check
        
        # 2) LLM-as-a-judge: strict True/False
        is_correct = self_evaluate(
            question=t["prompt"],
            prediction=got,
            expected_answer=t["expected"],
            model=judge_model,
        )
        is_correct = bool(is_correct)
        
        sample["history"]["check_correct1"]["self_eval"] = is_correct
        
        seperator('\nMODEL OUTPUT --> FIRST EVAL')
        print('match check:', match_check)
        print('self_eval:', is_correct)
        correctness, agreement = check_correct(match_check, is_correct)
        sample["history"]["check_correct1"]['correctness'] = correctness
        sample["history"]["check_correct1"]['agreement'] = agreement
        
        #starting and ending matches
        #CAN BE USED TO VALIDATE SECOND MODEL, OR AS LAST RESORT
        start_matches, end_matches = super_match(t, got)
        sides_matching = start_matches or end_matches
        
        if not agreement:
            #second model eval
            seperator('\nDISAGREEMENT --> SECOND EVAL')
            is_correct2 = self_evaluate2(
                question=t["prompt"],
                model_output=got,
                expected_answer=t["expected"],
                prediction=is_correct,
                model=judge_model
            )
            is_correct2 = bool(is_correct2)
            
            sample["history"]["check_correct2"]["self_eval"] = is_correct
            sample["history"]["check_correct2"]["self_eval2"] = is_correct2
            
            print('self_eval2:', is_correct2)
            correctness, agreement = check_correct(is_correct, is_correct2)
            sample["history"]["check_correct2"]["correctness"] = correctness
            sample["history"]["check_correct2"]["agreement"] = agreement
            
            
            if not agreement:
                #second model eval
                seperator('\nDISAGREEMENT --> THIRD EVAL')
                print('\nside matching:', sides_matching)
                
                sample["history"]["check_correct3"]["self_eval2"] = is_correct2
                sample["history"]["check_correct3"]["sides_matching"] = sides_matching
                correctness, agreement = check_correct(sides_matching, is_correct2)
                sample["history"]["check_correct3"]["correctness"] = correctness
                sample["history"]["check_correct3"]["agreement"] = agreement    


        sample["history"]["final_correctness"] = f"✅ {correctness}" if correctness else f"❌ {correctness}"
        final_answers.append(sample)
        
        if sleep_sec:
            time.sleep(sleep_sec)

    test_samples["samples"] = final_answers
    return test_samples

# Example:


---

# Second Final Project Implementation

---

## Math section

Importing arithmatic solver inference time algorithm from mini lab 5. This inference time agent is based off the one from mini lab 5, but also contains a ton of extra additions like more math functions and extra handling, as well as a custom inference loop.

In [90]:
import os, json, textwrap, re, time, ast, operator as op
import requests
import math
# --- PROVIDED: prompts ---
SYSTEM_AGENT2 = """You are a math tool-using agent.
You may do exactly ONE of the following in your reply:
1) CALCULATE: <arithmetic expression>
   - use only numbers, + - * / **, parentheses, and round(x, ndigits)
   - example: CALCULATE: round((3*2.49)*1.07, 2)
2) FINAL: <answer>
Return ONE line with the directive and value. No other text.
"""
SYSTEM_AGENT = """You are a math tool-using agent.

IMPORTANT: You must respond with EXACTLY ONE LINE in one of these formats:
1) CALCULATE: <arithmetic expression>
2) FINAL: <numeric answer>

Rules:
- NO explanations or reasoning
- NO LaTeX or markdown
- Arithmetic expressions can only use: numbers, +, -, *, /, **, (), round()
- Example valid responses:
  CALCULATE: (37 + 58) * 2
  CALCULATE: round(23.80 * 1.15, 2)
  FINAL: 190
  FINAL: 27.37

Respond with ONLY the directive line, nothing else.
"""

def make_first_prompt(question: str) -> str:
    return f"""Question: {question}
If you need arithmetic to get the answer, reply as:
CALCULATE: <expression>
Otherwise reply:
FINAL: <answer>"""

def make_second_prompt(result: str) -> str:
    return f"""The calculation result is: {result}
Now provide the final answer.
Reply exactly as: FINAL: <answer>"""


ACTION_RE = re.compile(r"^\s*(CALCULATE|FINAL)\s*:\s*(.+?)\s*$", re.IGNORECASE | re.DOTALL)


def parse_action(text: str):
    """
    Returns ("CALCULATE", expr) or ("FINAL", answer); raises ValueError on bad format.
    """
    # Take only the first line
    first_line = text.strip().split('\n')[0]
    m = ACTION_RE.match(first_line)
    if not m:
        raise ValueError(f"Unrecognized action format: {text!r}")
    action = m.group(1).upper()
    payload = m.group(2).strip()
    return action, payload

# We provide this function that evaluates arithmetic expressions.
ALLOWED_BINOPS = {ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul, ast.Div: op.truediv, ast.Pow: op.pow, ast.Mod: op.mod}
ALLOWED_UNOPS  = {ast.UAdd: op.pos, ast.USub: op.neg}

def handle_vars(expr: str, question: str):
    #SYSTEM_AGENT = f""

    new_prompt = f"""
                YOUR GOAL IS TO REPLACE VARIABLES WITH NUMBERS
                LETTERS ARE NOT ALLOWED IN THE EXPRESSION.
                
                A variable(s) was detected in this expression: {expr}
                
                Here is the original question: {question}
                
                Replace each variable (letter) with a number (to the best of your ability) and return ONLY the new expression. No explanation. No letters.
              """
    res = call_model_chat_completions(system=SYSTEM_AGENT, prompt=new_prompt)
    print("handle vars res:", res["text"])
    return ast.parse(res["text"], mode="eval")
    
    
def safe_eval(expr: str, question: str):
    """
    Evaluates a tiny arithmetic language: numbers, + - * / ** % parentheses, round(x, ndigits).
    Converts '^' to '**'. Rejects anything else.
    """
    expr = expr.replace("^", "**")
    expr = expr.replace("i", "j")
    if len(expr) > 300: #changed from 200 to 300.
        raise ValueError("Expression too long.")
    node = ast.parse(expr, mode="eval")

    count = 0
    def ev(n):
        if isinstance(n, ast.Expression):  return ev(n.body)
        if isinstance(n, ast.Constant) and isinstance(n.value, (int, float, complex)): return n.value
        if isinstance(n, ast.UnaryOp) and type(n.op) in ALLOWED_UNOPS:        return ALLOWED_UNOPS[type(n.op)](ev(n.operand))
        if isinstance(n, ast.BinOp) and type(n.op) in ALLOWED_BINOPS:         return ALLOWED_BINOPS[type(n.op)](ev(n.left), ev(n.right))
        if isinstance(n, ast.Call) and isinstance(n.func, ast.Name) and n.func.id in ["round", "abs", "sqrt", "ceil"]:
            args = [ev(a) for a in n.args]
            if n.func.id == "round":
                return round(*args)
            elif n.func.id == "abs":
                return abs(*args)
            elif n.func.id == "sqrt":
                return math.sqrt(*args)
            elif n.func.id == "ceil":
                return math.ceil(*args)
        if isinstance(n, ast.Tuple):  # allow round(x,2) with comma
            return tuple(ev(elt) for elt in n.elts)
        
        nonlocal count
        if count == 0:
            count += 1
            print(f"Disallowed expression: {ast.dump(n, include_attributes=False)}")
            return ev(handle_vars(expr, question))
        raise ValueError(f"Disallowed expression: {ast.dump(n, include_attributes=False)}")

    return ev(node)



def run_agent(question: str, max_tool_uses: int = 2, verbose: bool = True):
    # Turn 1
    r1 = call_model_chat_completions(system=SYSTEM_AGENT, prompt=make_first_prompt(question))
    if not r1["ok"]:
        raise RuntimeError(f"API error: {r1['error']}")
    if verbose: print("LLM →", r1["text"])
    action, payload = parse_action(r1["text"])

    tool_uses = 0
    while action == "CALCULATE":
        if tool_uses >= max_tool_uses:
            raise RuntimeError("Exceeded tool-use limit.")
        tool_uses += 1

        calc_value = safe_eval(payload, question) #pass it question so we can handle variables
        if verbose: print("CALC =", calc_value)

        # Turn 2 (+)
        rN = call_model_chat_completions(system=SYSTEM_AGENT, prompt=make_second_prompt(str(calc_value)))
        if not rN["ok"]:
            raise RuntimeError(f"API error: {rN['error']}")
        if verbose: print("LLM →", rN["text"])

        action, payload = parse_action(rN["text"])

    # action must be FINAL here
    return payload

Implementing the AST parser and math handling

In [133]:
import re, math

# ---------- Baseline: no-tool runner ----------
SYSTEM_DIRECT = "You are a careful math assistant. Reply with only the final numeric answer—no explanation."

NUM_RE = re.compile(r"[-+]?\d+(?:\.\d+)?(?:[eE][-+]?\d+)?")

def extract_number(text: str) -> str:
    """
    Try to normalize model output to a single numeric string.
    Falls back to the raw text if no number is found.
    """
    m = NUM_RE.search(text)
    num = m.group(0) if m else text.strip()
    #print(f"extracted num: {num}")
    return num

def make_direct_final_prompt(result: str) -> str:
    return f"""The calculation result is: {result}
            Now provide the final answer, no explanation.
            Reply exactly as: FINAL: <answer>"""

def run_direct(question: str, verbose: bool = True) -> str:
    r = call_model_chat_completions(
        system=SYSTEM_DIRECT,
        prompt=question,
        temperature=0.0,
        max_tokens=500
    )
    if not r["ok"]:
        raise RuntimeError(f"API error: {r['error']}")
    if verbose:
        print("LLM(no-tool) →", r["text"])
        
    r = call_model_chat_completions(
        system=SYSTEM_DIRECT,
        prompt=make_direct_final_prompt(r["text"]),
        temperature=0.0,
    )
    print(f"final direct: {r["text"]}")
    return extract_number(r["text"])

# ---------- If you need QUESTIONS / is_correct ----------
QUESTIONS = [
    ("What is (37 + 58) * 2?", "190"),
    ("A class has 28 students; 25% are absent. How many are present?", "21"),
    ("Solve 3x + 5 = 26 for x.", "7"),
    ("What is 12% of 240?", "28.8"),
    ("Average of 12, 18, 29, 31?", "22.5"),
    ("3 notebooks cost $2.49 each, plus 7% tax. Total to 2 decimals?", "7.99"),
    ("Convert 3.5 hours to minutes.", "210"),
    ("Perimeter of a rectangle 8 by 11.", "38"),
    ("What is 2.5^3?", "15.625"),
    ("Add a 15% tip to $23.80, round to 2 decimals.", "27.37"),
]

def is_correct(pred: str, gold: str):
    try:
        return abs(float(pred) - float(gold)) <= 1e-6
    except:
        return pred.strip() == gold.strip()

# ---------- Evaluation harness ----------
def evaluate_side_by_side(questions=QUESTIONS, verbose=False):
    rows = []
    direct_correct = 0
    tool_correct   = 0

    for i, (q, gold, ttype) in enumerate(questions, 1):
        gold = re.findall(r"\d+", gold)[-1] if len(gold) > 5 else gold
        
        if verbose: print(f"\nQ{i}: {q}")

        # No-tool
        pred_direct = run_direct(q, verbose=verbose)
        ok_direct   = is_correct(pred_direct, gold)
        direct_correct += int(ok_direct)

        # With tool
        pred_tool = run_agent(q, verbose=verbose)  # uses your agent loop
        ok_tool   = is_correct(pred_tool, gold)
        tool_correct += int(ok_tool)

        rows.append((i, q, gold, pred_direct, "✓" if ok_direct else "✗",
                               pred_tool,   "✓" if ok_tool   else "✗"))

    # Pretty print
    print("\n=== Results (No-Tool vs Tool) ===")
    colw = [4, 42, 8, 10, 3, 10, 3]
    header = ["#", "Question", "Gold", "No-Tool", "", "Tool", ""]
    fmt = f"{{:<{colw[0]}}} {{:<{colw[1]}}} {{:>{colw[2]}}}  {{:>{colw[3]}}} {{:^{colw[4]}}}  {{:>{colw[5]}}} {{:^{colw[6]}}}"
    print(fmt.format(*header))
    print("-" * sum(colw) + "-"*10)

    for r in rows:
        i, q, gold, pd, okd, pt, okt = r
        q_short = (q[:colw[1]-3] + "…") if len(q) > colw[1] else q
        print(fmt.format(i, q_short, gold, pd, okd, pt, okt))

    print(f"\nTotal (No-Tool): {direct_correct}/{len(questions)}")
    print(f"Total (Tool)   : {tool_correct}/{len(questions)}")

#evaluate_side_by_side(QUESTIONS, verbose=False)

## Search section (common_sense)

Various search util methods that aren't very useful

In [ ]:
from search_utils import search_bing, search_brave, search_duckduck

Wikipedia API

In [261]:
#%pip install wikipedia
import wikipedia

wikipedia.set_lang("en")
def search_wiki(query):
    query = query.strip()
    search_results = wikipedia.search(query, results=5)
    print(f"Search results: {search_results}")

    #first_result = search_results[0]
    
    try:
        summary = wikipedia.summary(query, sentences=3)
        print(f"Summary: {summary}")
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation needed: {e.options}")
    except wikipedia.exceptions.PageError:
        return f"Page not found for: {query}"
    

    #page = wikipedia.page(first_result)
    #pprint(page.content)
    """ print(f"Title: {page.title}")
    print(f"URL: {page.url}")
    print(f"Content length: {len(page.content)}")
    print(f"Images: {page.images[:3]}") """
    
    return summary

Main search agent/inference loop

In [243]:
def search_bot(question):
    print(question)

    SYSTEM_AGENT="""
    You are a helpful assistant that handles searching the web for more information.
    You respond with short and concise phrases.
    """
    
    prompt = f"""
    Do you want to research this question?
    
    QUESTION: {question}
    
    (Answer YES or NO)
    """
    
    prompt2 = f"""
    What wikipedia article should we research?
    
    QUESTION: {question}
    
    Only respond with a single article, don't explain.
    """
    
    prompt3 = f"""
    Given the wikipedia response given in the last chat. How would you answer this question:
    
    QUESTION: {question}
    
    Give a short concise response.
    """
    
    prompt4 = f"""
    If you don't want to search, then answer the question directly.
    
    QUESTION: {question}
    
    Give a short concise response.
    """
    
    r = call_model_chat_completions(
        system=SYSTEM_AGENT,
        prompt=prompt,
        temperature=0.2,
    )
    
    got = (r.get("text") or "").strip()
    print(got)
    
    if got == 'YES':
        chat = {
            "previous_chat": {
                "prompt": prompt,
                "your answer": got
            },
            "current_chat": {
                "prompt": prompt2,
                "your answer": "..."
            }
        }
    
        r = call_model_chat_completions(
            system=SYSTEM_AGENT,
            prompt=f"{chat}",
            temperature=0.2,
        )
        
        got = (r.get("text") or "").strip()
        print(got)
        chat["current_chat"]["your answer"] = got
        
        res = search_wiki(got)
        pprint(res)
        chat["current_chat"]["wikipedia response"] = res
        
        chat["previous_chat2"] = chat["current_chat"]
        del chat["current_chat"]
        
        chat["current_chat"] = {
                "prompt": prompt3,
                "your answer": "..."
            }
        print('chat', chat)
        r = call_model_chat_completions(
            system=SYSTEM_AGENT,
            prompt=f"{chat}",
            temperature=0.2,
        )
        
        got = (r.get("text") or "").strip()
        print(got)
        
    elif got == 'NO':
        r = call_model_chat_completions(
            system=SYSTEM_AGENT,
            prompt=prompt4,
            temperature=0.2,
        )
        
        got = (r.get("text") or "").strip()
        print(got)
        

## Coding question section

In [ ]:
def handle_test(test, output=None):
    display(Markdown(f"Expected Code:\n\n```python\n{test["expected"]}\n```"))
    if "```python" not in output:
        output = f"```python\n{output}\n```"
    return output


In [288]:
import re
import textwrap

def clean_code(code_str):
    code_str = re.sub(r"^```[a-zA-Z]*\n?", "", code_str)
    code_str = re.sub(r"\n?```$", "", code_str)

    code_str = code_str.encode().decode("unicode_escape")

    return textwrap.dedent(code_str).strip()


In [305]:
import subprocess
import textwrap
import sys

def handle_code(code):
    #code = textwrap.dedent(code)
    result = subprocess.run(
        [sys.executable, "-c", code],
        capture_output=True,
        text=True,
        timeout=50
    )

    print(result.stdout)
    print(result.stderr)
    
    return result.stdout, result.stderr

def get_code(test):
    input = test["prompt"]
    SYSTEM_AGENT = """
    You are a python developer who writes short and quick code.capitalize
    
    Don't forget to call your task func with real data at the end of the code. Print the result.
    """
    
    prompt1 = f"""
    Write the code to complete this question. Keep it short and sweet
    
    QUESTION: {input}
    ---------
    Call your task func with data at the end of the code to test it and print the result.
    """
    
    r = call_model_chat_completions(
        system=SYSTEM_AGENT,
        prompt=prompt1,
        temperature=0.2,
        max_tokens=350
    )
    got = (r.get("text") or "").strip()
    got = handle_test(test, got)
    display(Markdown(f"\n{got}"))
    
    print_json(got)
    cleaned = clean_code(got)
    handle_code(cleaned)
code = """
    import numpy as np

    def solve():
        x = np.array([1, 2, 3, 4])
        y = x ** 2
        return y.tolist()

    print(solve())
    """
#handle_code(code)


---

# Start loop

---

In [306]:
rng = 2393 #1741
#seed=11789, n=30 for diverse samples
test_prompts = get_tests(n=1, seed=rng, test_type=["coding"]) #get_test_type(["math"],end=10, upper=300) get_random_tests(n=3, upper=300)
print('seed:', rng)


QUESTIONS = []
for t in test_prompts:
    ttype = classify_prompt(t["prompt"])#guess_test_type(t["prompt"])
    print('test type:', ttype)
    print('question: ', t["prompt"])
    print('expected:', t["expected"])
    QUESTIONS.append((t["prompt"], t["expected"], ttype))

    if ttype == 'math':
        evaluate_side_by_side(QUESTIONS, verbose=True)
    elif ttype == 'common_sense':
        search_bot(t["prompt"])
    elif ttype == 'coding':
        get_code(t)
        """ parsed = handle_test(t["prompt"])
        display(Markdown(f"OUTPUT:\n{parsed}"))
        
        print_json(parsed) """

#load_save_json(path_in="test_history.json", path_out="test_history.json", data_in=f"seed: {rng}, questions: {test_prompts}", clear=False)
""" results_llm_judge = self_evaluate_tests(test_prompts, verbose=True, model=MODEL, grader_model=MODEL)
results_llm_judge["seed"] = rng
print_json(results_llm_judge)
print("\n","="*64)
load_save_json(path_in="test_history.json", path_out="test_history.json", data_in=results_llm_judge, clear=True) """


filtered size: 100
seed: 2393
prediction: coding
Classes order: ['coding' 'common_sense' 'future_prediction' 'math' 'planning']
probabilities: [9.98752478e-01 1.31286686e-04 6.83534731e-05 9.86201264e-04
 6.16805603e-05]
test type: coding
question:  Perform a chi-square test of independence of variables in a contingency table. This function takes a DataFrame containing categorical data and two column names, then constructs a contingency table from the two categorical columns and performs a chi-square test of independence. It returns the p-value of the test, which indicates the probability of observing the data if the null hypothesis (independence of the variables) is true. >>> np.random.seed(42) >>> data = pd.DataFrame({ ...     'a': np.random.choice(['A', 'B'], size=100), ...     'b': np.random.choice(['X', 'Y'], size=100) ... }) >>> task_func(data, 'a', 'b') 1.0
The function should raise the exception for: ValueError: If 'data' is empty, if 'col1' or 'col2' are not in 'data', if one 

Expected Code:

```python
    # Check if DataFrame is empty
    if data.empty:
        raise ValueError("The input DataFrame is empty.")

    # Check if specified columns exist
    if col1 not in data or col2 not in data:
        raise ValueError(f"One or both of the columns '{col1}' and '{col2}' do not exist in the DataFrame.")

    # Check for non-categorical data (numerical values)
    if np.issubdtype(data[col1].dtype, np.number) or np.issubdtype(data[col2].dtype, np.number):
        raise TypeError("One or both of the columns contain non-categorical data. The chi-square test requires categorical data.")

    # Check for single category (no variability)
    if len(data[col1].unique()) < 2 or len(data[col2].unique()) < 2:
        raise ValueError("One or both of the columns do not have multiple categories. The chi-square test requires variability in data.")

    # Check for small counts in numerous categories
    contingency_table = pd.crosstab(data[col1], data[col2])
    if (contingency_table < 5).any().any():
        raise ValueError("Some categories have less than 5 observations. This violates the assumptions of the chi-square test.")

    # Perform the chi-square test
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    return p
```


```python
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

def task_func(data, col1, col2):
    if data.empty or col1 not in data.columns or col2 not in data.columns:
        raise ValueError("Invalid input data or column names")
    if not (data[col1].dtype == 'object' and data[col2].dtype == 'object'):
        raise TypeError("Columns must contain categorical data")
    if len(data[col1].unique()) < 2 or len(data[col2].unique()) < 2:
        raise ValueError("Columns must have multiple categories")
    if data[col1].value_counts().min() < 5 or data[col2].value_counts().min() < 5:
        raise ValueError("Some categories have less than 5 observations")
    contingency = pd.crosstab(data[col1], data[col2])
    chi2, p, _, _ = chi2_contingency(contingency)
    return p

# Test with sample data
np.random.seed(42)
data = pd.DataFrame({
    'a': np.random.choice(['A', 'B'], size=100),
    'b': np.random.choice(['X', 'Y'], size=100)
})
print(task_func(data, 'a', 'b'))
```

"```python\nimport pandas as pd\nimport numpy as np\nfrom scipy.stats import chi2_contingency\n\ndef task_func(data, col1, col2):\n    if data.empty or col1 not in data.columns or col2 not in data.columns:\n        raise ValueError(\"Invalid input data or column names\")\n    if not (data[col1].dtype == 'object' and data[col2].dtype == 'object'):\n        raise TypeError(\"Columns must contain categorical data\")\n    if len(data[col1].unique()) < 2 or len(data[col2].unique()) < 2:\n        raise ValueError(\"Columns must have multiple categories\")\n    if data[col1].value_counts().min() < 5 or data[col2].value_counts().min() < 5:\n        raise ValueError(\"Some categories have less than 5 observations\")\n    contingency = pd.crosstab(data[col1], data[col2])\n    chi2, p, _, _ = chi2_contingency(contingency)\n    return p\n\n# Test with sample data\nnp.random.seed(42)\ndata = pd.DataFrame({\n    'a': np.random.choice(['A', 'B'], size=100),\n    'b': np.random.choice(['X', 'Y'], size

' results_llm_judge = self_evaluate_tests(test_prompts, verbose=True, model=MODEL, grader_model=MODEL)\nresults_llm_judge["seed"] = rng\nprint_json(results_llm_judge)\nprint("\n","="*64)\nload_save_json(path_in="test_history.json", path_out="test_history.json", data_in=results_llm_judge, clear=True) '

problem seeds: 1410, 12542 (NONE), 4260 (x), 8772 (choose), 1679(factorial)

15206 (wrong test), 18722(wiki cutting off n)

cool working example

```
seed: 4044
prediction: common_sense
Classes order: ['coding' 'common_sense' 'future_prediction' 'math' 'planning']
probabilities: [0.0938728  0.59071166 0.04256591 0.21505035 0.05779928]
test type: common_sense
expected: the Sumerians
Who invented the type of script used in autographs?
YES
Cursive script
Search results: ['Semi-cursive script', 'Cursive', 'Cursive script (East Asia)', 'Cursive Hebrew', 'Cursive script']
Summary: Semi-cursive script, also known as running script, is a style of Chinese calligraphy that emerged during the Han dynasty (202 BC – 220 AD). The style is used to write Chinese characters and is abbreviated slightly where a character's strokes are permitted to be visibly connected as the writer writes, but not to the extent of the cursive style. This makes the style easily readable by readers who can read regular script and quickly writable by calligraphers who require ideas to be written down quickly.
('Semi-cursive script, also known as running script, is a style of Chinese '
 'calligraphy that emerged during the Han dynasty (202 BC – 220 AD). The style '
 'is used to write Chinese characters and is abbreviated slightly where a '
 "character's strokes are permitted to be visibly connected as the writer "
 'writes, but not to the extent of the cursive style. This makes the style '
 'easily readable by readers who can read regular script and quickly writable '
 'by calligraphers who require ideas to be written down quickly.')
chat {'previous_chat': {'prompt': '\n    Do you want to research this question?\n\n    QUESTION: Who invented the type of script used in autographs?\n\n    (Answer YES or NO)\n    ', 'your answer': 'YES'}, 'previous_chat2': {'prompt': "\n    What wikipedia article should we research?\n\n    QUESTION: Who invented the type of script used in autographs?\n\n    Only respond with a single article, don't explain.\n    ", 'your answer': 'Cursive script', 'wikipedia response': "Semi-cursive script, also known as running script, is a style of Chinese calligraphy that emerged during the Han dynasty (202 BC – 220 AD). The style is used to write Chinese characters and is abbreviated slightly where a character's strokes are permitted to be visibly connected as the writer writes, but not to the extent of the cursive style. This makes the style easily readable by readers who can read regular script and quickly writable by calligraphers who require ideas to be written down quickly."}, 'current_chat': {'prompt': '\n    Given the wikipedia response given in the last chat. How would you answer this question:\n\n    QUESTION: Who invented the type of script used in autographs?\n\n    Give a short concise response.\n    ', 'your answer': '...'}}
The semi-cursive script, used in autographs, was developed during the Han dynasty.```